In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import shapiro
from statsmodels import robust
from scipy.spatial import distance


players = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/players.csv")
players

player_play = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/player_play.csv")
player_play

plays = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/plays.csv")
plays

motion = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/updated_motion_labels.csv")

plays_df = plays.dropna(subset=['pff_manZone'])

In [2]:
tracking1 = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_1.csv")
tracking2 = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_2.csv")
tracking3 = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_3.csv")
tracking4 = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_4.csv")
tracking5 = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_5.csv")
tracking6 = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_6.csv")
tracking7 = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_7.csv")
tracking8 = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_8.csv")
tracking9 = pd.read_csv("/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_9.csv")

dfs = [] 
file_list = ["/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_1.csv",
             "/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_2.csv",
             "/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_3.csv",
             "/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_4.csv",
             "/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_5.csv",
             "/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_6.csv",
             "/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_7.csv",
             "/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_8.csv",
             "/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/tracking_week_9.csv"]
for file in file_list:
    data = pd.read_csv(file)
    dfs.append(data) 
tracking_df = pd.concat(dfs, ignore_index=True)
tracking_df # Runtime: ~2m

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022091200,64,35459.0,Kareem Jackson,1,BEFORE_SNAP,2022-09-13 00:16:03.5,22.0,DEN,right,51.060000,28.550000,0.72,0.37,0.07,246.17,68.34,huddle_break_offense
1,2022091200,64,35459.0,Kareem Jackson,2,BEFORE_SNAP,2022-09-13 00:16:03.6,22.0,DEN,right,51.130000,28.570000,0.71,0.36,0.07,245.41,71.21,NaN
2,2022091200,64,35459.0,Kareem Jackson,3,BEFORE_SNAP,2022-09-13 00:16:03.7,22.0,DEN,right,51.200000,28.590000,0.69,0.23,0.07,244.45,69.90,NaN
3,2022091200,64,35459.0,Kareem Jackson,4,BEFORE_SNAP,2022-09-13 00:16:03.8,22.0,DEN,right,51.260000,28.620000,0.67,0.22,0.07,244.45,67.98,NaN
4,2022091200,64,35459.0,Kareem Jackson,5,BEFORE_SNAP,2022-09-13 00:16:03.9,22.0,DEN,right,51.320000,28.650000,0.65,0.34,0.07,245.74,62.83,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59327368,2022110300,3579,NaN,football,37,AFTER_SNAP,2022-11-04 03:04:35.5,NaN,football,left,74.830002,29.870001,0.00,0.00,0.00,NaN,NaN,qb_kneel
59327369,2022110300,3579,NaN,football,38,AFTER_SNAP,2022-11-04 03:04:35.6,NaN,football,left,74.830002,29.870001,0.00,0.00,0.00,NaN,NaN,NaN
59327370,2022110300,3579,NaN,football,39,AFTER_SNAP,2022-11-04 03:04:35.7,NaN,football,left,74.830002,29.870001,0.00,0.00,0.00,NaN,NaN,NaN
59327371,2022110300,3579,NaN,football,40,AFTER_SNAP,2022-11-04 03:04:35.8,NaN,football,left,74.830002,29.870001,0.00,0.00,0.00,NaN,NaN,NaN


In [3]:
pass_plays_df = plays_df.dropna(subset=['passResult'])
player_play

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,...,wasRunningRoute,routeRan,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_defensiveCoverageAssignment,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId
0,2022090800,56,35472,BUF,0,0,0,0,0,0,...,NaN,NaN,47917.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2022090800,56,42392,BUF,0,0,0,0,0,0,...,NaN,NaN,47917.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2022090800,56,42489,BUF,0,0,0,0,0,1,...,1.0,IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022090800,56,44875,BUF,0,0,0,0,0,0,...,NaN,NaN,43335.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,2022090800,56,44985,BUF,0,0,0,0,0,0,...,1.0,OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354722,2022110700,3787,52666,NO,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354723,2022110700,3787,52942,NO,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354724,2022110700,3787,53098,NO,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354725,2022110700,3787,54476,NO,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
pass_plays_df.columns
# player_play[player_play['pressureAllowedAsBlocker'] != np.nan]
player_play = player_play.rename(columns={'nflId': 'nflId_x'})
player_play

,gameId,playId,nflId_x,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,...,wasRunningRoute,routeRan,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_defensiveCoverageAssignment,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId
0,2022090800,56,35472,BUF,0,0,0,0,0,0,...,NaN,NaN,47917.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2022090800,56,42392,BUF,0,0,0,0,0,0,...,NaN,NaN,47917.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2022090800,56,42489,BUF,0,0,0,0,0,1,...,1.0,IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022090800,56,44875,BUF,0,0,0,0,0,0,...,NaN,NaN,43335.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,2022090800,56,44985,BUF,0,0,0,0,0,0,...,1.0,OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354722,2022110700,3787,52666,NO,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354723,2022110700,3787,52942,NO,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354724,2022110700,3787,53098,NO,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354725,2022110700,3787,54476,NO,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
motion[['gameId', 'playId']] = motion['game_play_id'].str.split(' ', expand=True)
motion

,game_play_id,motion_one,motion_two,motion_three,motion_four,motion_five,motion_six,gameId,playId
0,2022090800 101,0.0,0.0,0.0,0.0,0.0,1.0,2022090800,101
1,2022090800 1030,0.0,0.0,0.0,0.0,0.0,1.0,2022090800,1030
2,2022090800 1102,2.0,1.0,1.0,1.0,0.0,0.0,2022090800,1102
3,2022090800 1187,0.0,0.0,0.0,0.0,1.0,1.0,2022090800,1187
4,2022090800 1230,0.0,1.0,0.0,0.0,0.0,0.0,2022090800,1230
...,...,...,...,...,...,...,...,...,...
4917,2022110700 3059,1.0,0.0,0.0,0.0,0.0,0.0,2022110700,3059
4918,2022110700 3153,0.0,0.0,0.0,0.0,0.0,1.0,2022110700,3153
4919,2022110700 354,NaN,NaN,NaN,NaN,NaN,NaN,2022110700,354
4920,2022110700 588,1.0,0.0,0.0,0.0,0.0,0.0,2022110700,588


In [6]:
pass_plays_df.columns
player_play_subset = player_play[['gameId', 'playId', 'nflId_x', 'wasRunningRoute']]

play_data = tracking_df.merge(players, on='displayName', how='left')
play_data = play_data.drop(columns = ['collegeName', 'birthDate', 'weight', 'height', 'jerseyNumber'])
play_data_no_post_snap = play_data[play_data['frameType'] != 'AFTER_SNAP']

play_data_no_post_snap = play_data_no_post_snap.merge(player_play_subset, on = ['gameId', 'playId', 'nflId_x'], how='left')
play_data_no_post_snap['nflId_x'] = play_data_no_post_snap['nflId_x'].fillna(-1).astype(int)

play_data_no_post_snap # Runtime: ~2m

,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,y,s,a,dis,o,dir,event,nflId_y,position,wasRunningRoute
0,2022091200,64,35459,Kareem Jackson,1,BEFORE_SNAP,2022-09-13 00:16:03.5,DEN,right,51.060000,28.550000,0.72,0.37,0.07,246.17,68.34,huddle_break_offense,35459.0,SS,NaN
1,2022091200,64,35459,Kareem Jackson,2,BEFORE_SNAP,2022-09-13 00:16:03.6,DEN,right,51.130000,28.570000,0.71,0.36,0.07,245.41,71.21,NaN,35459.0,SS,NaN
2,2022091200,64,35459,Kareem Jackson,3,BEFORE_SNAP,2022-09-13 00:16:03.7,DEN,right,51.200000,28.590000,0.69,0.23,0.07,244.45,69.90,NaN,35459.0,SS,NaN
3,2022091200,64,35459,Kareem Jackson,4,BEFORE_SNAP,2022-09-13 00:16:03.8,DEN,right,51.260000,28.620000,0.67,0.22,0.07,244.45,67.98,NaN,35459.0,SS,NaN
4,2022091200,64,35459,Kareem Jackson,5,BEFORE_SNAP,2022-09-13 00:16:03.9,DEN,right,51.320000,28.650000,0.65,0.34,0.07,245.74,62.83,NaN,35459.0,SS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39635194,2022110300,3579,-1,football,22,BEFORE_SNAP,2022-11-04 03:04:34,football,left,74.800003,29.850000,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
39635195,2022110300,3579,-1,football,23,BEFORE_SNAP,2022-11-04 03:04:34.1,football,left,74.800003,29.850000,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
39635196,2022110300,3579,-1,football,24,BEFORE_SNAP,2022-11-04 03:04:34.2,football,left,74.800003,29.850000,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
39635197,2022110300,3579,-1,football,25,BEFORE_SNAP,2022-11-04 03:04:34.3,football,left,74.800003,29.860001,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
play_data_no_post_snap['gameId'] = play_data_no_post_snap['gameId'].astype(int)
motion['gameId'] = motion['gameId'].astype(int)
motion['playId'] = motion['playId'].astype(int)

play_data_no_post_snap = play_data_no_post_snap.merge(motion, on = ['gameId', 'playId'], how='left')

In [8]:
play_data_no_post_snap['wasRunningRoute'] = play_data_no_post_snap['wasRunningRoute'].fillna(0).astype(int)
play_data_no_post_snap['didBlock'] = abs(play_data_no_post_snap['wasRunningRoute'] - 1)

play_data_no_post_snap

,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,position,wasRunningRoute,game_play_id,motion_one,motion_two,motion_three,motion_four,motion_five,motion_six,didBlock
0,2022091200,64,35459,Kareem Jackson,1,BEFORE_SNAP,2022-09-13 00:16:03.5,DEN,right,51.060000,...,SS,0,2022091200 64,0.0,0.0,0.0,1.0,0.0,1.0,1
1,2022091200,64,35459,Kareem Jackson,2,BEFORE_SNAP,2022-09-13 00:16:03.6,DEN,right,51.130000,...,SS,0,2022091200 64,0.0,0.0,0.0,1.0,0.0,1.0,1
2,2022091200,64,35459,Kareem Jackson,3,BEFORE_SNAP,2022-09-13 00:16:03.7,DEN,right,51.200000,...,SS,0,2022091200 64,0.0,0.0,0.0,1.0,0.0,1.0,1
3,2022091200,64,35459,Kareem Jackson,4,BEFORE_SNAP,2022-09-13 00:16:03.8,DEN,right,51.260000,...,SS,0,2022091200 64,0.0,0.0,0.0,1.0,0.0,1.0,1
4,2022091200,64,35459,Kareem Jackson,5,BEFORE_SNAP,2022-09-13 00:16:03.9,DEN,right,51.320000,...,SS,0,2022091200 64,0.0,0.0,0.0,1.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39635194,2022110300,3579,-1,football,22,BEFORE_SNAP,2022-11-04 03:04:34,football,left,74.800003,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
39635195,2022110300,3579,-1,football,23,BEFORE_SNAP,2022-11-04 03:04:34.1,football,left,74.800003,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
39635196,2022110300,3579,-1,football,24,BEFORE_SNAP,2022-11-04 03:04:34.2,football,left,74.800003,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
39635197,2022110300,3579,-1,football,25,BEFORE_SNAP,2022-11-04 03:04:34.3,football,left,74.800003,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [9]:
play_data_no_post_snap['club'].unique()

array(['DEN', 'SEA', 'football', 'TB', 'DAL', 'TEN', 'NYG', 'MIN', 'GB',
       'LV', 'LAC', 'KC', 'ARI', 'JAX', 'WAS', 'NYJ', 'BAL', 'MIA', 'NE',
       'IND', 'HOU', 'PHI', 'DET', 'CIN', 'PIT', 'SF', 'CHI', 'CLE',
       'CAR', 'NO', 'ATL', 'BUF', 'LA'], dtype=object)

In [10]:
# (filtered_play_data['club'] == 'LV') | 
#                              (filtered_play_data['club'] == 'NYG') |
#                              (filtered_play_data['club'] == 'HOU')|
#                              (filtered_play_data['club'] == 'IND')|
#                              (filtered_play_data['club'] == 'LA')|

# teams = play_data_no_post_snap[
#                              ((play_data_no_post_snap['club'] == 'GB') 
#                               | (play_data_no_post_snap['club'] == 'KC')
#                               | (play_data_no_post_snap['club'] == 'football'))
#                              ]

gb_kc_game_ids = play_data_no_post_snap[
    play_data_no_post_snap['club'].isin(['GB', 'KC'])
]['gameId'].unique()

teams = play_data_no_post_snap[
    play_data_no_post_snap['gameId'].isin(gb_kc_game_ids)
]

teams

,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,position,wasRunningRoute,game_play_id,motion_one,motion_two,motion_three,motion_four,motion_five,motion_six,didBlock
822147,2022091112,62,38632,Kirk Cousins,1,BEFORE_SNAP,2022-09-11 20:26:16.3,MIN,right,25.030000,...,QB,0,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,1
822148,2022091112,62,38632,Kirk Cousins,2,BEFORE_SNAP,2022-09-11 20:26:16.4,MIN,right,25.050000,...,QB,0,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,1
822149,2022091112,62,38632,Kirk Cousins,3,BEFORE_SNAP,2022-09-11 20:26:16.5,MIN,right,25.080000,...,QB,0,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,1
822150,2022091112,62,38632,Kirk Cousins,4,BEFORE_SNAP,2022-09-11 20:26:16.6,MIN,right,25.130000,...,QB,0,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,1
822151,2022091112,62,38632,Kirk Cousins,5,BEFORE_SNAP,2022-09-11 20:26:16.7,MIN,right,25.190000,...,QB,0,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496781,2022110603,3886,-1,football,32,BEFORE_SNAP,2022-11-06 21:08:43.3,football,left,92.809998,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
38496782,2022110603,3886,-1,football,33,BEFORE_SNAP,2022-11-06 21:08:43.4,football,left,92.809998,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
38496783,2022110603,3886,-1,football,34,BEFORE_SNAP,2022-11-06 21:08:43.5,football,left,92.820000,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
38496784,2022110603,3886,-1,football,35,BEFORE_SNAP,2022-11-06 21:08:43.6,football,left,92.820000,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [11]:
def calculate_distance_to_football(data, wanted_position):
    """
    Calculate the distance of players in a specific position to the football for every frame.

    Parameters:
    - data (DataFrame): The input DataFrame containing columns ['gameId', 'playId', 'frameId', 'displayName', 'x', 'y', 'position']
    - wanted_position (str): The player position for which to calculate the distance to the football.

    Returns:
    - DataFrame: The input DataFrame with an added column 'distanceToFootball'.
    """
    # Initialize the distance column
    data['distanceToFootball'] = float(0)  # Default to 0 for players without the wanted position

    # Group by game, play, and frame
    grouped = data.groupby(['gameId', 'playId', 'frameId'])

    for (game_id, play_id, frame_id), group in grouped:
        # Find the football's position in the current frame
        football = group[group['displayName'] == 'football']
        if football.empty:
            continue

        football_x = football['x'].iloc[0]
        football_y = football['y'].iloc[0]

        # Calculate distance for players with the wanted position
        data.loc[group.index, 'distanceToFootball'] = group.apply(
            lambda player: np.sqrt((player['x'] - football_x) ** 2 + (player['y'] - football_y) ** 2)
            if player['position'] == wanted_position and player['displayName'] != 'football' and pd.notna(player['x']) and pd.notna(player['y'])
            else 0,
            axis=1
        )

    return data

updated_data = calculate_distance_to_football(teams, 'TE')
updated_data # Runtime: 9m 

/var/folders/8x/9brmr5z936dchlptv7w2kjzr0000gn/T/ipykernel_68933/3277041765.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['distanceToFootball'] = float(0)  # Default to 0 for players without the wanted position


,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,wasRunningRoute,game_play_id,motion_one,motion_two,motion_three,motion_four,motion_five,motion_six,didBlock,distanceToFootball
822147,2022091112,62,38632,Kirk Cousins,1,BEFORE_SNAP,2022-09-11 20:26:16.3,MIN,right,25.030000,...,0,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,1,0.0
822148,2022091112,62,38632,Kirk Cousins,2,BEFORE_SNAP,2022-09-11 20:26:16.4,MIN,right,25.050000,...,0,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,1,0.0
822149,2022091112,62,38632,Kirk Cousins,3,BEFORE_SNAP,2022-09-11 20:26:16.5,MIN,right,25.080000,...,0,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,1,0.0
822150,2022091112,62,38632,Kirk Cousins,4,BEFORE_SNAP,2022-09-11 20:26:16.6,MIN,right,25.130000,...,0,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,1,0.0
822151,2022091112,62,38632,Kirk Cousins,5,BEFORE_SNAP,2022-09-11 20:26:16.7,MIN,right,25.190000,...,0,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496781,2022110603,3886,-1,football,32,BEFORE_SNAP,2022-11-06 21:08:43.3,football,left,92.809998,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
38496782,2022110603,3886,-1,football,33,BEFORE_SNAP,2022-11-06 21:08:43.4,football,left,92.809998,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
38496783,2022110603,3886,-1,football,34,BEFORE_SNAP,2022-11-06 21:08:43.5,football,left,92.820000,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
38496784,2022110603,3886,-1,football,35,BEFORE_SNAP,2022-11-06 21:08:43.6,football,left,92.820000,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0


In [12]:
updated_data[updated_data['position'] == 'TE']

,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,wasRunningRoute,game_play_id,motion_one,motion_two,motion_three,motion_four,motion_five,motion_six,didBlock,distanceToFootball
823380,2022091112,62,45094,Johnny Mundt,1,BEFORE_SNAP,2022-09-11 20:26:16.3,MIN,right,25.72,...,1,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,0,5.810422
823381,2022091112,62,45094,Johnny Mundt,2,BEFORE_SNAP,2022-09-11 20:26:16.4,MIN,right,25.84,...,1,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,0,5.690879
823382,2022091112,62,45094,Johnny Mundt,3,BEFORE_SNAP,2022-09-11 20:26:16.5,MIN,right,25.97,...,1,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,0,5.567558
823383,2022091112,62,45094,Johnny Mundt,4,BEFORE_SNAP,2022-09-11 20:26:16.6,MIN,right,26.10,...,1,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,0,5.452064
823384,2022091112,62,45094,Johnny Mundt,5,BEFORE_SNAP,2022-09-11 20:26:16.7,MIN,right,26.24,...,1,2022091112 62,0.0,0.0,0.0,0.0,0.0,2.0,0,5.326200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496673,2022110603,3886,54642,James Mitchell,32,BEFORE_SNAP,2022-11-06 21:08:43.3,DET,left,93.58,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.877229
38496674,2022110603,3886,54642,James Mitchell,33,BEFORE_SNAP,2022-11-06 21:08:43.4,DET,left,93.57,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.836040
38496675,2022110603,3886,54642,James Mitchell,34,BEFORE_SNAP,2022-11-06 21:08:43.5,DET,left,93.56,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.802696
38496676,2022110603,3886,54642,James Mitchell,35,BEFORE_SNAP,2022-11-06 21:08:43.6,DET,left,93.55,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3.761516


In [13]:
def calculate_distance_to_teammate(data, wanted_position):
    """
    Calculate the distance of players in a specific position to their nearest teammate for every frame.
    
    Parameters:
    - data (DataFrame): The input DataFrame containing tracking data
    - wanted_position (str): The player position for which to calculate the distance
    
    Returns:
    - DataFrame: The input DataFrame with added columns 'distanceToTeammate' and 'nearestTeammatePosition'
    """
    data['distanceToTeammate'] = float(0)
    data['nearestTeammatePosition'] = None
    
    grouped = data.groupby(['gameId', 'playId', 'frameId'])
    
    for (game_id, play_id, frame_id), group in grouped:
        position_players = group[group['position'] == wanted_position]
        
        if position_players.empty:
            continue
            
        for _, player in position_players.iterrows():
            teammates = group[
                (group['club'] == player['club']) &
                (group['position'] != wanted_position) &
                (group['displayName'] != 'football') &
                pd.notna(group['x']) & 
                pd.notna(group['y'])
            ]

            if not teammates.empty:
                distances = teammates.apply(
                    lambda teammate: np.sqrt(
                        (teammate['x'] - player['x']) ** 2 + 
                        (teammate['y'] - player['y']) ** 2
                    ),
                    axis=1
                )
                
                min_distance = distances.min()
                nearest_teammate = teammates.iloc[distances.argmin()]
                
                data.loc[player.name, 'distanceToTeammate'] = min_distance
                data.loc[player.name, 'nearestTeammatePosition'] = nearest_teammate['position']
    
    return data

In [14]:
updated_data = calculate_distance_to_teammate(updated_data, 'TE')
updated_data

/var/folders/8x/9brmr5z936dchlptv7w2kjzr0000gn/T/ipykernel_68933/2110570453.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['distanceToTeammate'] = float(0)  # Default to 0 for players without the wanted position
/var/folders/8x/9brmr5z936dchlptv7w2kjzr0000gn/T/ipykernel_68933/2110570453.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['nearestTeammatePosition'] = None


,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,motion_one,motion_two,motion_three,motion_four,motion_five,motion_six,didBlock,distanceToFootball,distanceToTeammate,nearestTeammatePosition
822147,2022091112,62,38632,Kirk Cousins,1,BEFORE_SNAP,2022-09-11 20:26:16.3,MIN,right,25.030000,...,0.0,0.0,0.0,0.0,0.0,2.0,1,0.0,0.0,None
822148,2022091112,62,38632,Kirk Cousins,2,BEFORE_SNAP,2022-09-11 20:26:16.4,MIN,right,25.050000,...,0.0,0.0,0.0,0.0,0.0,2.0,1,0.0,0.0,None
822149,2022091112,62,38632,Kirk Cousins,3,BEFORE_SNAP,2022-09-11 20:26:16.5,MIN,right,25.080000,...,0.0,0.0,0.0,0.0,0.0,2.0,1,0.0,0.0,None
822150,2022091112,62,38632,Kirk Cousins,4,BEFORE_SNAP,2022-09-11 20:26:16.6,MIN,right,25.130000,...,0.0,0.0,0.0,0.0,0.0,2.0,1,0.0,0.0,None
822151,2022091112,62,38632,Kirk Cousins,5,BEFORE_SNAP,2022-09-11 20:26:16.7,MIN,right,25.190000,...,0.0,0.0,0.0,0.0,0.0,2.0,1,0.0,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496781,2022110603,3886,-1,football,32,BEFORE_SNAP,2022-11-06 21:08:43.3,football,left,92.809998,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,None
38496782,2022110603,3886,-1,football,33,BEFORE_SNAP,2022-11-06 21:08:43.4,football,left,92.809998,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,None
38496783,2022110603,3886,-1,football,34,BEFORE_SNAP,2022-11-06 21:08:43.5,football,left,92.820000,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,None
38496784,2022110603,3886,-1,football,35,BEFORE_SNAP,2022-11-06 21:08:43.6,football,left,92.820000,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,None


In [18]:
def calculate_distance_to_defender(data, wanted_position):
    """
    Calculate the distance of players in a specific position to their nearest defender for every frame.
    
    Parameters:
    - data (DataFrame): The input DataFrame containing tracking data
    - wanted_position (str): The player position for which to calculate the distance
    
    Returns:
    - DataFrame: The input DataFrame with added columns 'distanceToDefender' and 'nearestDefenderPosition'
    """
    data['distanceToDefender'] = float(0)
    data['nearestDefenderPosition'] = None
    
    grouped = data.groupby(['gameId', 'playId', 'frameId'])
    
    for (game_id, play_id, frame_id), group in grouped:
        position_players = group[group['position'] == wanted_position]
        
        if position_players.empty:
            continue
            
        for _, player in position_players.iterrows():
            
            defenders = group[
                (group['club'] != player['club']) &
                (group['club'] != 'football') & 
                pd.notna(group['x']) & 
                pd.notna(group['y'])
            ]
            
            if not defenders.empty:
                
                distances = defenders.apply(
                    lambda defender: np.sqrt(
                        (defender['x'] - player['x']) ** 2 + 
                        (defender['y'] - player['y']) ** 2
                    ),
                    axis=1
                )
                
                min_distance = distances.min()
                nearest_defender = defenders.iloc[distances.argmin()]
                
                data.loc[player.name, 'distanceToDefender'] = min_distance
                data.loc[player.name, 'nearestDefenderPosition'] = nearest_defender['position']
    
    return data


In [19]:
updated_data = calculate_distance_to_defender(updated_data, 'TE')
updated_data

/var/folders/8x/9brmr5z936dchlptv7w2kjzr0000gn/T/ipykernel_68933/2845413047.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['distanceToDefender'] = float(0)  # Default to 0 for players without the wanted position
/var/folders/8x/9brmr5z936dchlptv7w2kjzr0000gn/T/ipykernel_68933/2845413047.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['nearestDefenderPosition'] = None


,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,motion_three,motion_four,motion_five,motion_six,didBlock,distanceToFootball,distanceToTeammate,nearestTeammatePosition,distanceToDefender,nearestDefenderPosition
822147,2022091112,62,38632,Kirk Cousins,1,BEFORE_SNAP,2022-09-11 20:26:16.3,MIN,right,25.030000,...,0.0,0.0,0.0,2.0,1,0.0,0.0,None,0.0,None
822148,2022091112,62,38632,Kirk Cousins,2,BEFORE_SNAP,2022-09-11 20:26:16.4,MIN,right,25.050000,...,0.0,0.0,0.0,2.0,1,0.0,0.0,None,0.0,None
822149,2022091112,62,38632,Kirk Cousins,3,BEFORE_SNAP,2022-09-11 20:26:16.5,MIN,right,25.080000,...,0.0,0.0,0.0,2.0,1,0.0,0.0,None,0.0,None
822150,2022091112,62,38632,Kirk Cousins,4,BEFORE_SNAP,2022-09-11 20:26:16.6,MIN,right,25.130000,...,0.0,0.0,0.0,2.0,1,0.0,0.0,None,0.0,None
822151,2022091112,62,38632,Kirk Cousins,5,BEFORE_SNAP,2022-09-11 20:26:16.7,MIN,right,25.190000,...,0.0,0.0,0.0,2.0,1,0.0,0.0,None,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496781,2022110603,3886,-1,football,32,BEFORE_SNAP,2022-11-06 21:08:43.3,football,left,92.809998,...,NaN,NaN,NaN,NaN,1,0.0,0.0,None,0.0,None
38496782,2022110603,3886,-1,football,33,BEFORE_SNAP,2022-11-06 21:08:43.4,football,left,92.809998,...,NaN,NaN,NaN,NaN,1,0.0,0.0,None,0.0,None
38496783,2022110603,3886,-1,football,34,BEFORE_SNAP,2022-11-06 21:08:43.5,football,left,92.820000,...,NaN,NaN,NaN,NaN,1,0.0,0.0,None,0.0,None
38496784,2022110603,3886,-1,football,35,BEFORE_SNAP,2022-11-06 21:08:43.6,football,left,92.820000,...,NaN,NaN,NaN,NaN,1,0.0,0.0,None,0.0,None


In [20]:
updated_data[updated_data['position'] == 'TE']

,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,motion_three,motion_four,motion_five,motion_six,didBlock,distanceToFootball,distanceToTeammate,nearestTeammatePosition,distanceToDefender,nearestDefenderPosition
823380,2022091112,62,45094,Johnny Mundt,1,BEFORE_SNAP,2022-09-11 20:26:16.3,MIN,right,25.72,...,0.0,0.0,0.0,2.0,0,5.810422,0.506063,G,6.987797,NT
823381,2022091112,62,45094,Johnny Mundt,2,BEFORE_SNAP,2022-09-11 20:26:16.4,MIN,right,25.84,...,0.0,0.0,0.0,2.0,0,5.690879,0.678012,WR,6.810888,NT
823382,2022091112,62,45094,Johnny Mundt,3,BEFORE_SNAP,2022-09-11 20:26:16.5,MIN,right,25.97,...,0.0,0.0,0.0,2.0,0,5.567558,0.696348,WR,6.651474,NT
823383,2022091112,62,45094,Johnny Mundt,4,BEFORE_SNAP,2022-09-11 20:26:16.6,MIN,right,26.10,...,0.0,0.0,0.0,2.0,0,5.452064,0.718679,WR,6.522898,NT
823384,2022091112,62,45094,Johnny Mundt,5,BEFORE_SNAP,2022-09-11 20:26:16.7,MIN,right,26.24,...,0.0,0.0,0.0,2.0,0,5.326200,0.733553,WR,6.387378,NT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496673,2022110603,3886,54642,James Mitchell,32,BEFORE_SNAP,2022-11-06 21:08:43.3,DET,left,93.58,...,NaN,NaN,NaN,NaN,1,3.877229,1.126410,T,2.014051,OLB
38496674,2022110603,3886,54642,James Mitchell,33,BEFORE_SNAP,2022-11-06 21:08:43.4,DET,left,93.57,...,NaN,NaN,NaN,NaN,1,3.836040,1.113194,T,1.996121,OLB
38496675,2022110603,3886,54642,James Mitchell,34,BEFORE_SNAP,2022-11-06 21:08:43.5,DET,left,93.56,...,NaN,NaN,NaN,NaN,1,3.802696,1.116288,T,1.966647,OLB
38496676,2022110603,3886,54642,James Mitchell,35,BEFORE_SNAP,2022-11-06 21:08:43.6,DET,left,93.55,...,NaN,NaN,NaN,NaN,1,3.761516,1.100545,T,1.952076,OLB


In [21]:
updated_data_te = updated_data[updated_data['position'] == 'TE']

,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,motion_three,motion_four,motion_five,motion_six,didBlock,distanceToFootball,distanceToTeammate,nearestTeammatePosition,distanceToDefender,nearestDefenderPosition
823380,2022091112,62,45094,Johnny Mundt,1,BEFORE_SNAP,2022-09-11 20:26:16.3,MIN,right,25.72,...,0.0,0.0,0.0,2.0,0,5.810422,0.506063,G,6.987797,NT
823381,2022091112,62,45094,Johnny Mundt,2,BEFORE_SNAP,2022-09-11 20:26:16.4,MIN,right,25.84,...,0.0,0.0,0.0,2.0,0,5.690879,0.678012,WR,6.810888,NT
823382,2022091112,62,45094,Johnny Mundt,3,BEFORE_SNAP,2022-09-11 20:26:16.5,MIN,right,25.97,...,0.0,0.0,0.0,2.0,0,5.567558,0.696348,WR,6.651474,NT
823383,2022091112,62,45094,Johnny Mundt,4,BEFORE_SNAP,2022-09-11 20:26:16.6,MIN,right,26.10,...,0.0,0.0,0.0,2.0,0,5.452064,0.718679,WR,6.522898,NT
823384,2022091112,62,45094,Johnny Mundt,5,BEFORE_SNAP,2022-09-11 20:26:16.7,MIN,right,26.24,...,0.0,0.0,0.0,2.0,0,5.326200,0.733553,WR,6.387378,NT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496673,2022110603,3886,54642,James Mitchell,32,BEFORE_SNAP,2022-11-06 21:08:43.3,DET,left,93.58,...,NaN,NaN,NaN,NaN,1,3.877229,1.126410,T,2.014051,OLB
38496674,2022110603,3886,54642,James Mitchell,33,BEFORE_SNAP,2022-11-06 21:08:43.4,DET,left,93.57,...,NaN,NaN,NaN,NaN,1,3.836040,1.113194,T,1.996121,OLB
38496675,2022110603,3886,54642,James Mitchell,34,BEFORE_SNAP,2022-11-06 21:08:43.5,DET,left,93.56,...,NaN,NaN,NaN,NaN,1,3.802696,1.116288,T,1.966647,OLB
38496676,2022110603,3886,54642,James Mitchell,35,BEFORE_SNAP,2022-11-06 21:08:43.6,DET,left,93.55,...,NaN,NaN,NaN,NaN,1,3.761516,1.100545,T,1.952076,OLB


In [42]:
def determine_orientation(row):
    """
    Determines the player's orientation label based on 'o', 'playDirection', and 'y'.

    Parameters:
    - row (Series): A row of the DataFrame with columns 'o', 'playDirection', and 'y'.

    Returns:
    - str: The orientation label ('Backwards', 'Steep Inside', 'Inside', 'Forward', 'Outside', 'Steep Outside').
    """
    y_threshold = 26.65
    o = row['o']
    play_direction = row['playDirection']
    y = row['y']

    # Play direction: left
    if play_direction == 'left':
        if 0 <= o < 180:
            return 'Backwards'

        if y < y_threshold:
            if 360 >= o > 324:
                return 'Steep Inside'
            elif 324 >= o > 288:
                return 'Inside'
            elif 288 >= o > 252:
                return 'Forward'
            elif 252 >= o > 216:
                return 'Outside'
            elif 216 >= o > 180:
                return 'Steep Outside'
        
        else:
            if 360 >= o > 324:
                return 'Steep Outside'
            elif 324 >= o > 288:
                return 'Outside'
            elif 288 >= o > 252:
                return 'Forward'
            elif 252 >= o > 216:
                return 'Inside'
            elif 216 >= o > 180:
                return 'Steep Inside'

    # Play direction: right
    elif play_direction == 'right':
        if 180 < o <= 360:
            return 'Backwards'

        if y < y_threshold:
            if 0 <= o < 36:
                return 'Steep Inside'
            elif 36 <= o < 72:
                return 'Inside'
            elif 72 <= o < 108:
                return 'Forward'
            elif 108 <= o < 144:
                return 'Outside'
            elif 144 <= o < 180:
                return 'Steep Outside'
        
        else:
            if 0 <= o < 36:
                return 'Steep Outside'
            elif 36 <= o < 72:
                return 'Outside'
            elif 72 <= o < 108:
                return 'Forward'
            elif 108 <= o < 144:
                return 'Inside'
            elif 144 <= o < 180:
                return 'Steep Inside'

    return None  

updated_data_te['orientation'] = updated_data_te.apply(determine_orientation, axis=1)

In [43]:
updated_data_te

,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,motion_six,didBlock,distanceToFootball,distanceToTeammate,nearestTeammatePosition,distanceToDefender,nearestDefenderPosition,position_encoded,nearestTeammatePosition_encoded,orientation
823380,2022091112,62,45094,Johnny Mundt,1,BEFORE_SNAP,2022-09-11 20:26:16.3,MIN,right,25.72,...,2.0,0,5.810422,0.506063,G,6.987797,NT,0,3,Steep Outside
823381,2022091112,62,45094,Johnny Mundt,2,BEFORE_SNAP,2022-09-11 20:26:16.4,MIN,right,25.84,...,2.0,0,5.690879,0.678012,WR,6.810888,NT,0,7,Steep Outside
823382,2022091112,62,45094,Johnny Mundt,3,BEFORE_SNAP,2022-09-11 20:26:16.5,MIN,right,25.97,...,2.0,0,5.567558,0.696348,WR,6.651474,NT,0,7,Steep Outside
823383,2022091112,62,45094,Johnny Mundt,4,BEFORE_SNAP,2022-09-11 20:26:16.6,MIN,right,26.10,...,2.0,0,5.452064,0.718679,WR,6.522898,NT,0,7,Steep Outside
823384,2022091112,62,45094,Johnny Mundt,5,BEFORE_SNAP,2022-09-11 20:26:16.7,MIN,right,26.24,...,2.0,0,5.326200,0.733553,WR,6.387378,NT,0,7,Steep Outside
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496673,2022110603,3886,54642,James Mitchell,32,BEFORE_SNAP,2022-11-06 21:08:43.3,DET,left,93.58,...,0.0,1,3.877229,1.126410,T,2.014051,OLB,0,6,Forward
38496674,2022110603,3886,54642,James Mitchell,33,BEFORE_SNAP,2022-11-06 21:08:43.4,DET,left,93.57,...,0.0,1,3.836040,1.113194,T,1.996121,OLB,0,6,Forward
38496675,2022110603,3886,54642,James Mitchell,34,BEFORE_SNAP,2022-11-06 21:08:43.5,DET,left,93.56,...,0.0,1,3.802696,1.116288,T,1.966647,OLB,0,6,Forward
38496676,2022110603,3886,54642,James Mitchell,35,BEFORE_SNAP,2022-11-06 21:08:43.6,DET,left,93.55,...,0.0,1,3.761516,1.100545,T,1.952076,OLB,0,6,Forward


In [44]:
data_gb_kc = updated_data_te[(updated_data_te['club'] == 'KC') | (updated_data_te ['club'] == 'GB')]
data_gb_kc

,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,motion_six,didBlock,distanceToFootball,distanceToTeammate,nearestTeammatePosition,distanceToDefender,nearestDefenderPosition,position_encoded,nearestTeammatePosition_encoded,orientation
849446,2022091112,336,30842,Marcedes Lewis,1,BEFORE_SNAP,2022-09-11 20:37:30.4,GB,left,91.31,...,1.0,0,6.197242,0.757694,G,8.007877,DT,0,3,Inside
849447,2022091112,336,30842,Marcedes Lewis,2,BEFORE_SNAP,2022-09-11 20:37:30.5,GB,left,91.24,...,1.0,0,6.163320,0.865390,G,7.971148,DT,0,3,Forward
849448,2022091112,336,30842,Marcedes Lewis,3,BEFORE_SNAP,2022-09-11 20:37:30.6,GB,left,91.15,...,1.0,0,6.114001,0.974987,G,7.917228,DT,0,3,Forward
849449,2022091112,336,30842,Marcedes Lewis,4,BEFORE_SNAP,2022-09-11 20:37:30.7,GB,left,91.04,...,1.0,0,6.062387,1.072241,G,7.864890,DT,0,3,Forward
849450,2022091112,336,30842,Marcedes Lewis,5,BEFORE_SNAP,2022-09-11 20:37:30.8,GB,left,90.93,...,1.0,0,6.002969,1.179534,G,7.804800,DT,0,3,Forward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38494236,2022110603,3857,45635,Robert Tonyan,97,BEFORE_SNAP,2022-11-06 21:07:50.6,GB,right,90.81,...,0.0,0,6.274074,2.332831,T,2.837464,DE,0,6,Forward
38494237,2022110603,3857,45635,Robert Tonyan,98,BEFORE_SNAP,2022-11-06 21:07:50.7,GB,right,90.81,...,0.0,0,6.274074,2.332831,T,2.828162,DE,0,6,Forward
38494238,2022110603,3857,45635,Robert Tonyan,99,BEFORE_SNAP,2022-11-06 21:07:50.8,GB,right,90.81,...,0.0,0,6.274074,2.342691,T,2.822570,DE,0,6,Forward
38494239,2022110603,3857,45635,Robert Tonyan,100,BEFORE_SNAP,2022-11-06 21:07:50.9,GB,right,90.81,...,0.0,0,6.232985,2.360784,T,2.804015,DE,0,6,Forward


In [47]:
# updated_data['playDirection'].replace(['left', 'right'],[0, 1], inplace=True)
updated_data_te = updated_data_te.fillna(int(0))
updated_data_te['nearestTeammatePosition'] = updated_data_te['nearestTeammatePosition'].astype(str)
updated_data_te['nearestDefenderPosition'] = updated_data_te['nearestDefenderPosition'].astype(str)

In [48]:
from sklearn.preprocessing import LabelEncoder

updated_data_te['position_encoded'] = LabelEncoder().fit_transform(updated_data_te['position'])
updated_data_te['nearestTeammatePosition'] = updated_data_te['nearestTeammatePosition'].astype(str)
updated_data_te['nearestTeammatePosition_encoded'] = LabelEncoder().fit_transform(updated_data_te['nearestTeammatePosition'])
updated_data_te['orientation_encoded'] = LabelEncoder().fit_transform(updated_data_te['orientation'])
updated_data_te['nearestDefenderPosition_encoded'] = LabelEncoder().fit_transform(updated_data_te['nearestDefenderPosition'])

In [ ]:
# final_processed_data['closest_teammate_distance'] = final_processed_data['closest_teammate_distance'].fillna(-1)
# final_processed_data['closest_teammate_position_encoded'] = final_processed_data['closest_teammate_position_encoded'].fillna(-1)
# final_processed_data['closest_defender_position_encoded'] = final_processed_data['closest_defender_position_encoded'].fillna(-1)
# final_processed_data['closest_defender_distance'] = final_processed_data['closest_defender_distance'].fillna(-1)
# final_processed_data.isnull().any()

In [49]:
# Step 1: Identify unique gameIds and sort them
unique_game_ids = updated_data['gameId'].unique()
unique_game_ids.sort()

# Step 2: Specify the gameIds to include in the testing set
specific_test_game_ids = [2022100900]  # Replace with your desired gameIds

# Step 3: Exclude the specific test gameIds from the unique gameIds for splitting
remaining_game_ids = [gid for gid in unique_game_ids if gid not in specific_test_game_ids]

# Step 4: Determine the split point for 75% training and 25% testing on remaining gameIds
split_index = int(len(remaining_game_ids) * 0.75)
train_game_ids = remaining_game_ids[:split_index]
test_game_ids = remaining_game_ids[split_index:]

# Step 5: Add the specific test gameIds to the test set
test_game_ids.extend(specific_test_game_ids)

# Step 6: Create training and testing datasets based on gameIds
train = updated_data_te[updated_data_te['gameId'].isin(train_game_ids)]
test = updated_data_te[updated_data_te['gameId'].isin(test_game_ids)]

test[test['gameId'] == 2022100900]


,gameId,playId,nflId_x,displayName,frameId,frameType,time,club,playDirection,x,...,distanceToFootball,distanceToTeammate,nearestTeammatePosition,distanceToDefender,nearestDefenderPosition,position_encoded,nearestTeammatePosition_encoded,orientation,orientation_encoded,nearestDefenderPosition_encoded
22657173,2022100900,66,30842,Marcedes Lewis,1,BEFORE_SNAP,2022-10-09 13:33:30.2,GB,left,90.77,...,5.170155,0.947523,G,7.023425,OLB,0,3,Forward,1,8
22657174,2022100900,66,30842,Marcedes Lewis,2,BEFORE_SNAP,2022-10-09 13:33:30.3,GB,left,90.59,...,5.093625,0.990051,G,6.828250,OLB,0,3,Forward,1,8
22657175,2022100900,66,30842,Marcedes Lewis,3,BEFORE_SNAP,2022-10-09 13:33:30.4,GB,left,90.41,...,5.014539,1.039663,G,6.638200,OLB,0,3,Forward,1,8
22657176,2022100900,66,30842,Marcedes Lewis,4,BEFORE_SNAP,2022-10-09 13:33:30.5,GB,left,90.22,...,4.954081,1.068316,G,6.415801,OLB,0,3,Forward,1,8
22657177,2022100900,66,30842,Marcedes Lewis,5,BEFORE_SNAP,2022-10-09 13:33:30.6,GB,left,90.03,...,4.896337,1.111261,G,6.199177,OLB,0,3,Forward,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993359,2022100900,3858,45635,Robert Tonyan,134,BEFORE_SNAP,2022-10-09 16:31:41.4,GB,left,75.12,...,6.935143,3.783755,G,2.300543,OLB,0,3,Forward,1,8
22993360,2022100900,3858,45635,Robert Tonyan,135,BEFORE_SNAP,2022-10-09 16:31:41.5,GB,left,75.11,...,6.934011,3.786093,G,2.298282,OLB,0,3,Forward,1,8
22993361,2022100900,3858,45635,Robert Tonyan,136,BEFORE_SNAP,2022-10-09 16:31:41.6,GB,left,75.11,...,6.934011,3.776374,G,2.298282,OLB,0,3,Forward,1,8
22993362,2022100900,3858,45635,Robert Tonyan,137,BEFORE_SNAP,2022-10-09 16:31:41.7,GB,left,75.12,...,6.935143,3.774030,G,2.310087,OLB,0,3,Forward,1,8


In [ ]:
test[test['position'] == 'TE']
test = test.fillna(0)

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


X = test[[ 's','a', 'dis', 
                          'motion_one', 'motion_two', 'motion_three', 'motion_four',
                          'motion_five', 'motion_six', 'distanceToFootball',
                          'position_encoded', 'nearestTeammatePosition_encoded', 
                          'orientation_encoded', 'nearestDefenderPosition_encoded',
                          'distanceToDefender', 'distanceToTeammate']]
y = test['didBlock']  

X_train = train[[ 's','a', 'dis', 
                          'motion_one', 'motion_two', 'motion_three', 'motion_four',
                          'motion_five', 'motion_six', 'distanceToFootball',
                          'position_encoded', 'nearestTeammatePosition_encoded', 
                          'orientation_encoded', 'nearestDefenderPosition_encoded',
                          'distanceToDefender', 'distanceToTeammate']]
X_test = test[[ 's','a', 'dis', 
                          'motion_one', 'motion_two', 'motion_three', 'motion_four',
                          'motion_five', 'motion_six', 'distanceToFootball',
                          'position_encoded', 'nearestTeammatePosition_encoded', 
                          'orientation_encoded', 'nearestDefenderPosition_encoded',
                          'distanceToDefender', 'distanceToTeammate']]
y_train = train['didBlock']
y_test = test['didBlock']

In [52]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_train_rnn = np.expand_dims(X_train.values, axis=1)
X_test_rnn = np.expand_dims(X_test.values, axis=1)  

model = Sequential()

model.add(SimpleRNN(64, activation='relu', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=False))

model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_rnn, y_train, epochs=10, batch_size=32, validation_data=(X_test_rnn, y_test))

y_pred = model.predict(X_test_rnn)

test['prediction'] = pd.Series(y_pred.ravel(), index=test.index)

y_pred_binary = (y_pred > 0.5).astype(int)

print('Accuracy:', accuracy_score(y_test, y_pred_binary))
print('Classification Report:')
print(classification_report(y_test, y_pred_binary))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_binary))

def permutation_importance_rnn(model, X_test, y_test, baseline_accuracy):
    """
    Calculate permutation feature importance for an RNN model.
    
    Parameters:
        model: Trained RNN model.
        X_test: Test dataset (3D array: samples, time steps, features).
        y_test: True labels for the test set.
        baseline_accuracy: Accuracy of the model on the unpermuted test set.
    
    Returns:
        A list of accuracy drops (importance scores) for each feature.
    """
    importance_scores = []
    for col in range(X_test.shape[2]): 
        X_test_permuted = X_test.copy()
        np.random.shuffle(X_test_permuted[:, 0, col])
        
        y_pred_permuted = model.predict(X_test_permuted)
        y_pred_binary_permuted = (y_pred_permuted > 0.5).astype(int)
        
        permuted_accuracy = accuracy_score(y_test, y_pred_binary_permuted)
        
        importance_scores.append(baseline_accuracy - permuted_accuracy)
    
    return importance_scores

baseline_accuracy = accuracy_score(y_test, y_pred_binary)

importance_scores = permutation_importance_rnn(model, X_test_rnn, y_test, baseline_accuracy)

feature_importance = {col: score for col, score in zip(X_train.columns, importance_scores)}

sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

print("\nFeature Importance (Drop in Accuracy):")
for feature, score in sorted_importance:
    print(f"{feature}: {score:.4f}")

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5960/5960 ━━━━━━━━━━━━━━━━━━━━ 5s 810us/step - accuracy: 0.6363 - loss: 0.6469 - val_accuracy: 0.6835 - val_loss: 0.6200
Epoch 2/10
5960/5960 ━━━━━━━━━━━━━━━━━━━━ 5s 833us/step - accuracy: 0.6807 - loss: 0.5974 - val_accuracy: 0.6760 - val_loss: 0.6177
Epoch 3/10
5960/5960 ━━━━━━━━━━━━━━━━━━━━ 5s 847us/step - accuracy: 0.6850 - loss: 0.5898 - val_accuracy: 0.6711 - val_loss: 0.6197
Epoch 4/10
5960/5960 ━━━━━━━━━━━━━━━━━━━━ 5s 828us/step - accuracy: 0.6897 - loss: 0.5834 - val_accuracy: 0.6749 - val_loss: 0.6276
Epoch 5/10
5960/5960 ━━━━━━━━━━━━━━━━━━━━ 5s 764us/step - accuracy: 0.6930 - loss: 0.5787 - val_accuracy: 0.6789 - val_loss: 0.6263
Epoch 6/10
5960/5960 ━━━━━━━━━━━━━━━━━━━━ 5s 773us/step - accuracy: 0.6948 - loss: 0.5758 - val_accuracy: 0.6789 - val_loss: 0.6345
Epoch 7/10
5960/5960 ━━━━━━━━━━━━━━━━━━━━ 5s 779us/step - accuracy: 0.6992 - loss: 0.5722 - val_accuracy: 0.6791 - val_loss: 0.6390
Epoch 8/10
5960/5960 ━━━━━━━━━━━━━━━━━━━━ 5s 784us/step - accuracy: 0.7003 - loss: 0.56

/var/folders/8x/9brmr5z936dchlptv7w2kjzr0000gn/T/ipykernel_68933/146973922.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['prediction'] = pd.Series(y_pred.ravel(), index=test.index)


2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 262us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 296us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 288us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 256us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 278us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 295us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 271us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 264us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 274us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 273us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 338us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 328us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 284us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 269us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 345us/step
2757/2757 ━━━━━━━━━━━━━━━━━━━━ 1s 264us/step

Feature Importance (Drop in Accuracy):
distanceToFootball: 0.0495
distanceToTeammate: 0.0465
motion_six: 0.0256
s: 0.0244
motion_one: 0.0203
motion_three: 0.0185
distanceToDefender: 0.0126
a: 0.0062
orientation_encoded: 0.0055
nearestDefenderPosition_encoded: 0.0054
motion_four:

In [ ]:
test[(test['gameId'] == 2022100900) & (test['playId'] == 1329)]
updated_data[(updated_data['gameId'] == 2022100900) & (test['playId'] == 1329)]

In [54]:
# game_id = 2022091105
# play_id = 962
# test[(test['gameId'] == game_id) & (test['playId'] == play_id)].tail(50) 


game_id = 2022110603
play_id = 534


data_2022110603_534 = updated_data[(updated_data['gameId'] == game_id) & (updated_data['playId'] == play_id)]
data_2022110603_534 = data_2022110603_534.merge(
    test[['position', 'prediction']],
    on='position',
    how='left'
)

data_2022110603_534.to_csv('data_2022110603_534.csv', index=False)
data_2022110603_534_te = data_2022110603_534[data_2022110603_534['position'] == 'TE']
data_2022110603_534_te.to_csv('data_2022110603_534_te.csv', index=False)
data_2022110603_534_te

OSError: [Errno 28] No space left on device

: 

In [ ]:
# Animation Visualizer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio
import numpy as np
import pandas as pd
from scipy.spatial import distance
from IPython.display import display, Image

# Input values
frame = 153
game_id = 2022100900
play_id = 736

# Filter data for the specific play and frame
play299 = tracking_df[
    (tracking_df['gameId'] == game_id) &
    (tracking_df['playId'] == play_id)
]

play299_ball = tracking_df[
    (tracking_df['gameId'] == game_id) &
    (tracking_df['playId'] == play_id) &
    (tracking_df['frameId'] == frame) &
    (tracking_df['club'] == 'football')
]

# Merge play data with players information
play299 = play299.merge(players, on='displayName', how='left')

# Separate offense and defense
offense = play299[play299['club'] == play299[play299['position'] == 'QB']['club'].iloc[0]]
defense = play299[play299['club'] == play299[play299['position'] == 'CB']['club'].iloc[0]]

# Filter offense and defense for the specific frame
play299_offense = offense[offense['frameId'] == frame]
play299_defense = defense[defense['frameId'] == frame]

# Load the background image
background_img = mpimg.imread('/Users/danielsoriano/Downloads/nfl-big-data-bowl-2025/football_field.jpg')

# Define target positions and initialize frames for the GIF
target_positions = {'TE': 2}
gif_frames = []

# Loop through each frame to create the animation
for frame_id in sorted(play299['frameId'].unique()):
    frame_data = play299[play299['frameId'] == frame_id]
    
    # Identify clubs (teams) excluding the football
    clubs = frame_data['club'].unique()
    clubs = [club for club in clubs if club != 'football'][:2]
    
    # Separate offensive and defensive players in the frame
    off_players_frame = frame_data[frame_data['club'] == clubs[0]]
    def_players = frame_data[frame_data['club'] == clubs[1]]
    
    # Filter target players (e.g., Tight Ends)
    off_targets = off_players_frame[off_players_frame['position'].isin(target_positions.keys())]
    off_target_data = []
    for position, count in target_positions.items():
        position_players = off_targets[off_targets['position'] == position]
        off_target_data.append(position_players.head(count))
    off_target_data = pd.concat(off_target_data)

    tightend_data = frame_data[frame_data['position'] == 'TE']

    # Create a plot for the frame
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.set_xlim(0, 120)
    ax.set_ylim(0, 53.3)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title(f'Frame {frame_id}')
    ax.imshow(background_img, extent=[0, 120, 0, 53.3], aspect='auto', alpha=0.9)
    ax.set_xticks(range(0, 121, 10))  
    ax.grid(color='gray', linestyle='--', linewidth=0.5, axis='x')  
    
    # Plot players and ball
    ax.scatter(def_players['x'], def_players['y'], color='blue', s=55, label=clubs[1])
    ax.scatter(off_players_frame['x'], off_players_frame['y'], color='yellow', s=55, label=clubs[0])
    ax.scatter(tightend_data['x'], tightend_data['y'], color='orange', s=55, label='Tight End')
    ax.scatter(frame_data[frame_data['club'] == 'football']['x'], 
               frame_data[frame_data['club'] == 'football']['y'], 
               color='brown', s=35, label='Football', marker='d')

    # Draw lines between offensive players and their closest defense/teammates
    for _, off_player in off_target_data.iterrows():
        distances = def_players.apply(
            lambda def_player: distance.euclidean(
                (off_player['x'], off_player['y']),
                (def_player['x'], def_player['y'])
            ),
            axis=1
        )
        closest_def_index = distances.idxmin()
        closest_def_player = def_players.loc[closest_def_index]

        teammate_distances = off_players_frame[off_players_frame.index != off_player.name].apply(
            lambda teammate: distance.euclidean(
                (off_player['x'], off_player['y']),
                (teammate['x'], teammate['y'])
            ),
            axis=1
        )
        closest_teammate_index = teammate_distances.idxmin()
        closest_teammate = off_players_frame.loc[closest_teammate_index]

        # ax.plot(
        #     [off_player['x'], closest_def_player['x']], 
        #     [off_player['y'], closest_def_player['y']], 
        #     color='red', linewidth=2
        # )
        ax.plot(
            [off_player['x'], closest_teammate['x']], 
            [off_player['y'], closest_teammate['y']], 
            color='purple', linewidth=2
        )

    ax.legend(loc='upper right')

    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    gif_frames.append(image)
    plt.close(fig)

gif_path = 'players_positions_with_background.gif'
imageio.mimsave(gif_path, gif_frames, duration=0.1)

display(Image(filename=gif_path))
